In [1]:
import os
import sys
sys.path.append(os.pardir)

In [2]:
import json
import datetime
import pandas as pd
from sklearn import preprocessing
from credible import connectors

In [3]:
engine = connectors.connect_to_sqlite()

In [4]:
%%time
users_raw = pd.read_sql_table('users', engine)
reviews_raw = pd.read_sql_table('reviews', engine)

CPU times: user 1min 14s, sys: 31.5 s, total: 1min 46s
Wall time: 2min 53s


## Cleaning on Users

In [61]:
reviews_raw.sample(3)

,_id,review_id,business_id,user_id,stars,date,text,useful,funny,cool
2291175,2291176,LtCX7oJ0ixOX7dCQHZHXLQ,LgroNa-JH5W-wpJACXuz5A,dKavJNhDwJjKAe7rDLV6xw,1,2014-06-17 04:12:09,"I love conveyor sushi joints, and while this o...",1,0,0
1026041,1026042,nDTDnN6j7b6Lv_LE9WGwCQ,z24D8Qgf4yW5SU1_nPVsCA,maugsnfDmeRareS8EFu1SA,5,2015-12-30 21:32:01,"Excellent value for the price. A small, family...",1,0,0
977923,977924,Skxlg6u3ijoJ3EoKmbdrsw,Jjwa-WCK73efAwVZRksWSw,Fjk2oT1hqy-790SOqlJpFg,5,2018-05-15 01:12:47,"I saw ""POTs"" announced on 'VEGANS BABY' FB sit...",1,0,2


In [25]:
users_raw.average_stars.apply(lambda x: x != 3).value_counts()

True     1108108
False     529030
Name: average_stars, dtype: int64

In [68]:
ind_rev_count = users_raw.review_count.apply(lambda x: (int(x) < 20) & (int(x) > 5))
ind_friends = users_raw.friends.apply(lambda x: len(x.split(',')) < 10)
ind_star = users_raw.average_stars.apply(lambda x: x != 3)

df_users_clean = users_raw[ind_friends & ind_rev_count & ind_star]
df_users_clean.shape

(166781, 23)

In [73]:
df_users_clean.user_id.to_sql('users_clean', con=engine, index=False, if_exists='replace')
pd.read_sql_table('users_clean', con=engine).sample(3)

,user_id
117521,IJ3JPdGCyMprS0sacQojXw
36374,VI4gZBDFqqQq2ldjSFcCTw
70424,d0jMop-CoA0B6aNdMheZGw


## Cleaning on Reviews

In [69]:
ind_rev_leng = reviews_raw.text.apply(lambda x: len(x) < 200)
ind_cool = reviews_raw.cool.apply(lambda x: x < 5)
ind_funny = reviews_raw.funny.apply(lambda x: x < 5)
ind_useful = reviews_raw.useful.apply(lambda x: x < 5)

df_reviews_clean = reviews_raw[ind_rev_leng & ind_cool & ind_funny & ind_useful]
df_reviews_clean.shape

(1245408, 10)

In [72]:
df_reviews_clean.review_id.to_sql('reviews_clean', con=engine, index=False, if_exists='replace')
pd.read_sql_table('reviews_clean', con=engine).sample(3)

,review_id
772935,uJdfpdhSBdO38TGhxVfvyg
1040310,2zMMVEzH2uCh5UAayxlB5A
538473,tSbsNO7mpPIZT2RI0bUIUQ
